In [1]:
#This notebook builds a heirarchial model from baseline randomforest 
#Morgan fingerprint 
#and cell painting

In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [3]:
list_of_lists_df = pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble.csv")
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.648612,1,0.317810,0.794860,1,0.290626,1.0,0.282609,0.858368,Training,True,True,0.742456,0.855408,1.0,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.261878,0,0.317810,0.587383,1,0.290626,1.0,0.576923,0.710612,Training,False,True,0.412004,0.709168,1.0,True
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.025140,0,0.317810,0.180529,0,0.290626,0.0,0.727273,0.492026,Training,True,True,0.039553,0.310587,0.0,True
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.018895,0,0.317810,0.230988,0,0.290626,0.0,0.800000,0.717828,Training,True,True,0.029727,0.397398,0.0,True
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.258888,0,0.317810,0.116859,0,0.290626,0.0,0.738462,0.457042,Training,True,True,0.407301,0.201047,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,"InChI=1S/C8H11ClN2O3S2/c1-4(2)16(13,14)5-3-15-...",0.309551,1,0.240921,0.322350,1,0.260921,0.0,0.250000,0.618373,HeldOut,False,False,0.545206,0.541558,1.0,False
58483,93_259,InChI=1S/C18H23NO3/c1-13(12-22-17-6-4-3-5-7-17...,0.267934,1,0.240921,0.223786,0,0.260921,0.0,0.391304,0.603436,HeldOut,False,True,0.517793,0.428839,0.0,True
58484,93_259,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.335675,1,0.240921,0.283991,1,0.260921,1.0,0.277778,0.826193,HeldOut,True,True,0.562413,0.515607,1.0,True
58485,93_259,InChI=1S/C16H13N3O3/c1-22-16(21)19-15-17-12-8-...,0.334415,1,0.240921,0.371684,1,0.260921,0.0,0.344828,0.763229,HeldOut,False,False,0.561583,0.574934,1.0,False


In [4]:
from sklearn.linear_model import LogisticRegression
from io import StringIO
from itertools import product
import sys
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

In [5]:
def RF_Fn(assay):

    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    
    
    proba_batch=[]
    pred_batch=[]
    true_batch=[]
    StdInChI_batch=[]
    
    train= detail_list[detail_list["Data"]=="Training"]
    
    #Sample Train data
    train_inactive = train[train["true"] == 0]
    inactives_n=len(train_inactive)
    print("Inactive Compounds ", inactives_n)

    train_active = train[train["true"] == 1]
    actives_n=len(train_active)
    print("Active Compounds ", actives_n)

    if(inactives_n > actives_n):
            train_inactive= train_inactive.sample(actives_n, random_state=1)
            print("Inactive Compounds (after Undersamplimg) ", len(train_inactive) )
            train= train_active.append(train_inactive) 
            print("Total Compounds (after Undersamplimg) ", len(train) )

    elif(actives_n > inactives_n):
            train_active= train_active.sample(inactives_n, random_state=1)
            print("Active Compounds (after Undersamplimg) ", len(train_active) )
            train= train_inactive.append(train_active) 
            print("Total Compounds (after Undersamplimg) ", len(train) )

    
    #test
    test= detail_list[detail_list["Data"]=="HeldOut"]
    

    print("Herirachal model for training data")    

    print("LR Model")
            
    X_train = train[["fp_proba_scaled", "CP_proba_scaled"]].to_numpy()
    y_train = train["true"].to_numpy()
    X_test = test[["fp_proba_scaled", "CP_proba_scaled"]].to_numpy()
    y_test = test["true"].to_numpy()


    classifier = LogisticRegression(C=5, random_state=42)            
    classifier.fit(X_train, y_train)

           
    StdInChI = test["InChICode_standardised"].to_numpy()
    pred = classifier.predict(X_test) 
    proba = classifier.predict_proba(X_test)[:,1]
    true = test["true"].to_numpy()
            

    StdInChI_batch = np.concatenate([StdInChI_batch, StdInChI])
    proba_batch = np.concatenate([proba_batch, proba])
    pred_batch = np.concatenate([pred_batch, pred])
    true_batch = np.concatenate([true_batch, true])

    print(len(pred_batch))
    print(len(true_batch))
   
    return StdInChI_batch, true_batch, pred_batch, proba_batch

In [6]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

def check_heirarchial(assay, method, y_true, y_pred, y_prob):
    
    conf_matrix = confusion_matrix(y_true, y_pred)
    #print(conf_matrix)
    
    #print(classification_report(y_true, y_pred))
    
    ba= balanced_accuracy_score(y_true, y_pred)
    #print('balanced_accuracy_score ',ba)
    
    mcc=matthews_corrcoef(y_true, y_pred)
    #print('matthews_corrcoef ',mcc)
    # Sensitivity, hit rate, recall, or true positive rate
    
    f1= f1_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    
    #print('F1 Toxic', f1)
    #print('precision', precision)
    
    Specificity = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[0,1])
    # Specificity or true negative rate
    Sensitivity = conf_matrix[1,1]/(conf_matrix[1,0]+conf_matrix[1,1]) 
    
    #print( 'Sensitivity', Sensitivity)
    #print( 'Specificity', Specificity)
    AUC = roc_auc_score(y_true, y_prob)
    #print('AUC-ROC ',AUC)    
    # calculate roc curves
    best_model_fpr, best_model_tpr, _ = roc_curve(y_true, y_prob)   
    
    
    row=[assay, method, f1, precision, Sensitivity, Specificity, ba, mcc, AUC]
    metrics_list.append(row)
    
    return metrics_list

In [7]:
import warnings
warnings.filterwarnings("ignore") #ignore, default
import pandas

#sns.set(rc={'figure.figsize':(2, 2)})
#plt.rcParams['figure.dpi'] = 80
#sns.set_style("white")
metrics_list=[]
all_preds_test_compounds=pd.DataFrame(columns=['assay', 'InChICode_standardised', 'fp_proba', 'fp_pred', 'fp_threshold', 'CP_proba',
       'CP_pred', 'CP_threshold', 'true', 'ts', 'pc', 'Data', 'MFP_Correct',
       'CP_Correct', 'fp_proba_scaled', 'CP_proba_scaled', 'simple_pred',
       'Ensemble_Correct', 'heirarchial_pred', 'heirarchial_Correct'])


#for assay in tqdm(assaylist[:7]):
for assay in tqdm(list_of_lists_df.assay.unique()[:]):
    
    print("Assay: ", assay)
    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    detail_list_test = detail_list[detail_list["Data"]=="HeldOut"]

    print("Heirarchial_Model")
    merger_df=pd.DataFrame()
    
    StdInChI_batch, true_batch, pred_batch, proba_batch = RF_Fn(assay)
    check_heirarchial(assay, "Heirarchial Model", true_batch, pred_batch, proba_batch)
    
    #print(true_batch)
    
    print(assay)
    
    merger_df["InChICode_standardised"]=StdInChI_batch
    merger_df["heirarchial_pred"]= pred_batch
    merger_df["true"]=true_batch
    merger_df["heirarchial_Correct"]= merger_df["heirarchial_pred"] == merger_df["true"]
    merger_df["assay"]=assay
    
    #.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="herrachial_Correct", legend=False)
    
    #For each assay combine original predictions and merger_df predictions
    #all_preds = pd.DataFrame()
    all_preds_test_compounds= pd.concat([all_preds_test_compounds, pd.merge(detail_list_test, merger_df)])  
    
    
    

  4%|█▉                                          | 4/89 [00:00<00:02, 36.63it/s]

Assay:  178_684
Heirarchial_Model
Inactive Compounds  163
Active Compounds  54
Inactive Compounds (after Undersamplimg)  54
Total Compounds (after Undersamplimg)  108
Herirachal model for training data
LR Model
55
55
178_684
Assay:  180_686
Heirarchial_Model
Inactive Compounds  122
Active Compounds  41
Inactive Compounds (after Undersamplimg)  41
Total Compounds (after Undersamplimg)  82
Herirachal model for training data
LR Model
41
41
180_686
Assay:  183_604
Heirarchial_Model
Inactive Compounds  108
Active Compounds  36
Inactive Compounds (after Undersamplimg)  36
Total Compounds (after Undersamplimg)  72
Herirachal model for training data
LR Model
36
36
183_604
Assay:  184_606
Heirarchial_Model
Inactive Compounds  187
Active Compounds  62
Inactive Compounds (after Undersamplimg)  62
Total Compounds (after Undersamplimg)  124
Herirachal model for training data
LR Model
63
63
184_606
Assay:  185_665
Heirarchial_Model
Inactive Compounds  72
Active Compounds  24
Inactive Compounds (afte

 13%|█████▊                                     | 12/89 [00:00<00:02, 38.26it/s]

190_628
Assay:  190_629
Heirarchial_Model
Inactive Compounds  62
Active Compounds  21
Inactive Compounds (after Undersamplimg)  21
Total Compounds (after Undersamplimg)  42
Herirachal model for training data
LR Model
21
21
190_629
Assay:  190_674
Heirarchial_Model
Inactive Compounds  74
Active Compounds  25
Inactive Compounds (after Undersamplimg)  25
Total Compounds (after Undersamplimg)  50
Herirachal model for training data
LR Model
25
25
190_674
Assay:  191_630
Heirarchial_Model
Inactive Compounds  110
Active Compounds  37
Inactive Compounds (after Undersamplimg)  37
Total Compounds (after Undersamplimg)  74
Herirachal model for training data
LR Model
37
37
191_630
Assay:  191_631
Heirarchial_Model
Inactive Compounds  98
Active Compounds  33
Inactive Compounds (after Undersamplimg)  33
Total Compounds (after Undersamplimg)  66
Herirachal model for training data
LR Model
33
33
191_631
Assay:  193_723
Heirarchial_Model
Inactive Compounds  120
Active Compounds  40
Inactive Compounds (

 22%|█████████▋                                 | 20/89 [00:00<00:01, 36.75it/s]

...

Heirarchial_Model
Inactive Compounds  88
Active Compounds  30
Inactive Compounds (after Undersamplimg)  30
Total Compounds (after Undersamplimg)  60
Herirachal model for training data
LR Model
30
30
264_742
Assay:  265_743
Heirarchial_Model
Inactive Compounds  1838
Active Compounds  613
Inactive Compounds (after Undersamplimg)  613
Total Compounds (after Undersamplimg)  1226
Herirachal model for training data
LR Model
613
613
265_743
Assay:  266_744
Heirarchial_Model
Inactive Compounds  417
Active Compounds  139
Inactive Compounds (after Undersamplimg)  139
Total Compounds (after Undersamplimg)  278
Herirachal model for training data
LR Model
140
140
266_744
Assay:  267_745
Heirarchial_Model
Inactive Compounds  544
Active Compounds  182
Inactive Compounds (after Undersamplimg)  182
Total Compounds (after Undersamplimg)  364
Herirachal model for training data
LR Model
182
182
267_745
Assay:  268_746
Heirarchial_Model
Inactive Compounds  952
Active Compounds  318
Inactive Compounds (afte

 94%|████████████████████████████████████████▌  | 84/89 [00:02<00:00, 37.44it/s]

190
190
271_749
Assay:  272_750
Heirarchial_Model
Inactive Compounds  184
Active Compounds  62
Inactive Compounds (after Undersamplimg)  62
Total Compounds (after Undersamplimg)  124
Herirachal model for training data
LR Model
62
62
272_750
Assay:  273_751
Heirarchial_Model
Inactive Compounds  108
Active Compounds  36
Inactive Compounds (after Undersamplimg)  36
Total Compounds (after Undersamplimg)  72
Herirachal model for training data
LR Model
36
36
273_751
Assay:  274_752
Heirarchial_Model
Inactive Compounds  256
Active Compounds  86
Inactive Compounds (after Undersamplimg)  86
Total Compounds (after Undersamplimg)  172
Herirachal model for training data
LR Model
86
86
274_752
Assay:  275_753
Heirarchial_Model
Inactive Compounds  93
Active Compounds  31
Inactive Compounds (after Undersamplimg)  31
Total Compounds (after Undersamplimg)  62
Herirachal model for training data
LR Model
32
32
275_753
Assay:  275_754
Heirarchial_Model
Inactive Compounds  252
Active Compounds  84
Inactive

100%|███████████████████████████████████████████| 89/89 [00:02<00:00, 37.05it/s]

279_758
Assay:  280_759
Heirarchial_Model
Inactive Compounds  324
Active Compounds  108
Inactive Compounds (after Undersamplimg)  108
Total Compounds (after Undersamplimg)  216
Herirachal model for training data
LR Model
108
108
280_759
Assay:  283_762
Heirarchial_Model
Inactive Compounds  256
Active Compounds  86
Inactive Compounds (after Undersamplimg)  86
Total Compounds (after Undersamplimg)  172
Herirachal model for training data
LR Model
86
86
283_762
Assay:  93_259
Heirarchial_Model
Inactive Compounds  74
Active Compounds  27
Inactive Compounds (after Undersamplimg)  27
Total Compounds (after Undersamplimg)  54
Herirachal model for training data
LR Model
26
26
93_259


In [8]:
all_preds_test_compounds

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct,heirarchial_pred,heirarchial_Correct
0,178_684,InChI=1S/C24H28N4O4S/c1-14-12-28(15(2)13-29)24...,0.296817,0,0.325363,0.706426,1,0.240378,1.0,0.661017,0.731399,HeldOut,False,True,0.456133,0.806763,1.0,True,1.0,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.335199,1,0.325363,0.167642,0,0.240378,0.0,0.735294,0.447463,HeldOut,False,True,0.507290,0.348705,0.0,True,1.0,False
2,178_684,InChI=1S/C25H35N3O5S/c1-17-10-12-20(13-11-17)3...,0.110413,0,0.325363,0.190223,0,0.240378,0.0,0.785714,0.864848,HeldOut,True,True,0.169677,0.395675,0.0,True,0.0,True
3,178_684,InChI=1S/C31H42N4O6/c1-20-15-35(21(2)18-36)30(...,0.170428,0,0.325363,0.134469,0,0.240378,0.0,0.767123,0.750539,HeldOut,True,True,0.261904,0.279703,0.0,True,0.0,True
4,178_684,InChI=1S/C30H41N5O5/c1-20-17-35(21(2)19-36)28(...,0.231296,0,0.325363,0.079269,0,0.240378,1.0,0.591549,0.420531,HeldOut,False,False,0.355443,0.164885,0.0,False,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,93_259,"InChI=1S/C8H11ClN2O3S2/c1-4(2)16(13,14)5-3-15-...",0.309551,1,0.240921,0.322350,1,0.260921,0.0,0.250000,0.618373,HeldOut,False,False,0.545206,0.541558,1.0,False,1.0,False
22,93_259,InChI=1S/C18H23NO3/c1-13(12-22-17-6-4-3-5-7-17...,0.267934,1,0.240921,0.223786,0,0.260921,0.0,0.391304,0.603436,HeldOut,False,True,0.517793,0.428839,0.0,True,1.0,False
23,93_259,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.335675,1,0.240921,0.283991,1,0.260921,1.0,0.277778,0.826193,HeldOut,True,True,0.562413,0.515607,1.0,True,1.0,True
24,93_259,InChI=1S/C16H13N3O3/c1-22-16(21)19-15-17-12-8-...,0.334415,1,0.240921,0.371684,1,0.260921,0.0,0.344828,0.763229,HeldOut,False,False,0.561583,0.574934,1.0,False,1.0,False


In [9]:
all_preds_test_compounds.to_csv("Predictions_train_heldout_scaled_prob_all_assays_heirarchial_model_test_compounds.csv", index=False)

In [10]:
metrics_list = pd.DataFrame(metrics_list,columns=["assay","method","f1", "precision", "Sensitivity", "Specificity", "ba", "mcc", "AUC"])
metrics_list

,assay,method,f1,precision,Sensitivity,Specificity,ba,mcc,AUC
0,178_684,Heirarchial Model,0.620690,0.600000,0.642857,0.853659,0.748258,0.485639,0.789199
1,180_686,Heirarchial Model,0.516129,0.380952,0.800000,0.580645,0.690323,0.327022,0.716129
2,183_604,Heirarchial Model,0.418605,0.264706,1.000000,0.074074,0.537037,0.140028,0.720165
3,184_606,Heirarchial Model,0.700000,0.583333,0.875000,0.787234,0.831117,0.593584,0.921543
4,185_665,Heirarchial Model,0.352941,0.272727,0.500000,0.555556,0.527778,0.048280,0.564815
...,...,...,...,...,...,...,...,...,...
84,278_757,Heirarchial Model,0.544643,0.409396,0.813333,0.612335,0.712834,0.367854,0.802584
85,279_758,Heirarchial Model,0.478261,0.333333,0.846154,0.450000,0.648077,0.262874,0.815385
86,280_759,Heirarchial Model,0.733333,0.666667,0.814815,0.864198,0.839506,0.638285,0.942844
87,283_762,Heirarchial Model,0.250000,0.169492,0.476190,0.246154,0.361172,-0.257019,0.358242


In [11]:
metrics_list.to_csv("heirarchial_model_metrics.csv", index=False)